In [10]:
from newspaper import Article
from bs4 import BeautifulSoup
import requests
import urllib

#Hindu Business Line RSS Feed
url = "https://www.financialexpress.com/industry/banking-finance/feed/"

resp = requests.get(url)

bsObj = BeautifulSoup(resp.content, features = "xml")
links = bsObj.findAll('link', text=True)

##Links extracted from RSS Feed
print('EXTRACTED LINKS FROM RSS FEED')
for link in links:
    if(link.get_text()=="https://www.financialexpress.com"):
        links.remove(link)

for link in links:
    print(link.get_text())

EXTRACTED LINKS FROM RSS FEED
https://www.financialexpress.com
https://www.financialexpress.com/industry/banking-finance/jp-morgan-targets-mid-sized-firms-in-challenge-to-european-banks/1404684/
https://www.financialexpress.com/industry/banking-finance/nbfcs-asset-growth-to-halve-to-10-in-h2-on-liquidity-woes-says-crisil-report/1404413/
https://www.financialexpress.com/industry/banking-finance/outlook-on-banks-negative-private-lenders-to-recover-faster-psu-banks-need-this-much-capital-fitch/1403914/
https://www.financialexpress.com/industry/banking-finance/sbi-cease-to-be-epfo-fund-manager-from-march-end-says-labour-minister-santosh-gangwar/1403545/
https://www.financialexpress.com/industry/banking-finance/rbi-needs-to-relook-ownership-norms-for-pvt-lenders-impeding-expansion-of-large-financial-institutions-says-report/1403394/
https://www.financialexpress.com/industry/banking-finance/fusion-microfinance-raises-rs-520-cr-funding-from-warburg-pincus-others/1403370/
https://www.financial

In [11]:
from sqlalchemy import *
db = create_engine('sqlite:///FinancialDB.db', echo=True)

metadata = MetaData(db)

news = Table('news', metadata,
            Column('id', Integer, primary_key=True),
            Column('paperName', String(100)),
            Column('title', String(250)),
            Column('summary', String(10000))
            )
news.create()

2018-12-05 23:24:18,924 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-12-05 23:24:18,925 INFO sqlalchemy.engine.base.Engine ()
2018-12-05 23:24:18,927 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-12-05 23:24:18,928 INFO sqlalchemy.engine.base.Engine ()
2018-12-05 23:24:18,932 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE news (
	id INTEGER NOT NULL, 
	"paperName" VARCHAR(100), 
	title VARCHAR(250), 
	summary VARCHAR(10000), 
	PRIMARY KEY (id)
)


2018-12-05 23:24:18,933 INFO sqlalchemy.engine.base.Engine ()
2018-12-05 23:24:19,084 INFO sqlalchemy.engine.base.Engine COMMIT


In [12]:
#Using pagerank and textrank to get the summary

from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [13]:
import numpy as np
from nltk.tokenize import sent_tokenize
from operator import itemgetter 
import warnings
warnings.filterwarnings("ignore")

#Store all text in list
article_text = []
article_title = []
for link in links:
    article = Article(link.getText(), language="en")
    article.download()
    article.parse()
    if(len(article.text.split('.'))<=3):
        continue
    article_text.append(article.text)
    article_title.append(article.title)

stop_words = stopwords.words('english')

def build_similarity_matrix(sentences, stopwords=None):
    
    # Create an empty similarity matrix
    S = np.zeros((len(sentences), len(sentences)))
 
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
 
            S[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
 
    # normalize the matrix row-wise
    
    for idx in range(len(S)):
        if(S[idx].sum()==0):
            break
        S[idx] /= S[idx].sum()
 
    return S

def pagerank(A, eps=0.0001, d=0.85):
    P = np.ones(len(A)) / len(A)
    while True:
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs(new_P - P).sum()
        if delta <= eps:
            return new_P
        P = new_P


counter=0
temp=""
i = news.insert()
for texts in article_text:
    sentences = sent_tokenize(texts)
    S = build_similarity_matrix(sentences, stop_words)  
    sentence_ranks = pagerank(S)
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    SUMMARY_SIZE = 3
    SELECTED_SENTENCES = sorted(ranked_sentence_indexes[:SUMMARY_SIZE])
    summary = itemgetter(*SELECTED_SENTENCES)(sentences)
    # Print the actual summary
    print('\nSUMMARY')
    print('------------------------------')
    for sentence in summary:
        temp+=''.join(sentence) 
        print(''.join(sentence))
    counter+=1
    i.execute({'paperName':'Financial times', 'title':article_title[counter],'summary':temp})
    


SUMMARY
------------------------------
JP Morgan’s challenge to the European banks in their traditional stronghold is another example of the U.S. bank using its clout to try to take business from competitors, after holding up much better than many during the financial crisis.
Petno’s prospect list was two years in the making and built by those responsible for JP Morgan’s affairs in each country.
Petno declined to reveal his goals, but JP Morgan’s expansion comes as corporate lending is a rare bright spot for Europe’s banks amid rock-bottom interest rates and weak growth.
2018-12-05 23:26:02,967 INFO sqlalchemy.engine.base.Engine INSERT INTO news ("paperName", title, summary) VALUES (?, ?, ?)
2018-12-05 23:26:02,968 INFO sqlalchemy.engine.base.Engine ('Financial times', 'NBFCs’ asset growth to halve to 10% in H2 on liquidity woes, says Crisil report', 'JP Morgan’s challenge to the European banks in their traditional stronghold is another example of the U.S. bank using its clout to try 

2018-12-05 23:26:05,314 INFO sqlalchemy.engine.base.Engine COMMIT

SUMMARY
------------------------------
IDBI Bank Tuesday said government will not participate in LIC’s open offer to acquire 51 per cent stake in the bank.
Subsequently, LIC then came out with an open offer for acquiring 26 per cent of equity in IDBI Bank at Rs 61.73 per share.
The Insurance Regulatory and Development Authority of India in June had permitted LIC to increase its stake from 10.82 per cent to 51 per cent in the IDBI Bank.
2018-12-05 23:26:05,559 INFO sqlalchemy.engine.base.Engine INSERT INTO news ("paperName", title, summary) VALUES (?, ?, ?)
2018-12-05 23:26:05,560 INFO sqlalchemy.engine.base.Engine ('Financial times', 'ICICI Bank hikes MCLRs by 10 bps across tenures', 'JP Morgan’s challenge to the European banks in their traditional stronghold is another example of the U.S. bank using its clout to try to take busine ... (3496 characters truncated) ... ce Regulatory and Development Authority of India in J

2018-12-05 23:26:07,355 INFO sqlalchemy.engine.base.Engine ('Financial times', 'RBI likely to keep interest rate unchanged on December 5', 'JP Morgan’s challenge to the European banks in their traditional stronghold is another example of the U.S. bank using its clout to try to take busine ... (7656 characters truncated) ... dium enterprises.“The government should also allow the RBI to implement the Prompt Corrective Action (PCA) framework for less strong banks,” he said.')
2018-12-05 23:26:07,381 INFO sqlalchemy.engine.base.Engine COMMIT

SUMMARY
------------------------------
However, India’s economic growth slowed to 7.1 per cent in the September quarter after peaking to an over two-year high in the first three months of this fiscal, as consumption demand moderated and farm sector displayed signs of weakness.
Devendra Kumar Pant, Chief Economist, India Ratings and Research (Fitch Group), said 2018-19 may still end up with a GDP growth of 7.3 per cent and “RBI may get the much needed 

2018-12-05 23:26:09,834 INFO sqlalchemy.engine.base.Engine ('Financial times', "Yes Bank's Rana Kapoor unlikely to contest for Chairman job", 'JP Morgan’s challenge to the European banks in their traditional stronghold is another example of the U.S. bank using its clout to try to take busine ... (11674 characters truncated) ...  way, where banks get off at the first sign of trouble, from the earlier approach of tolerance and rehabilitate an account to protect the enterprise.')
2018-12-05 23:26:09,837 INFO sqlalchemy.engine.base.Engine COMMIT

SUMMARY
------------------------------
Chawla, a former bureaucrat, had resigned from the post of non-executive chairman, which, according to sources, was triggered in the wake of controversy on his continuance on the board of the bank while his name appeared in a CBI chargesheet in the Aircel-Maxis case.
Also, one new member — Uttam Prakash Agarwal — has joined the board, taking the current strength to seven.
Also, Yes Bank’s Search and Selection

2018-12-05 23:26:11,660 INFO sqlalchemy.engine.base.Engine COMMIT

SUMMARY
------------------------------
(PTI)

For all the cordiality on show at the end of Reserve Bank of India’s last board meeting, tensions could re-surface when it next gathers on December 14 as the government aims to ramp up pressure for changes in the way the central bank operates.
Like the government, most board members feel that RBI should be “more transparent and accountable” for its decisions, said one source, who was also aware of discussions held subsequently within the government.
That decision had followed on from the October board meeting, which noted an improvement among some banks under the PCA, based on parameters including bad loan recovery, higher deposit growth.
2018-12-05 23:26:12,564 INFO sqlalchemy.engine.base.Engine INSERT INTO news ("paperName", title, summary) VALUES (?, ?, ?)
2018-12-05 23:26:12,565 INFO sqlalchemy.engine.base.Engine ('Financial times', 'Yes Bank share price crashes 10% in 2

...

2018-12-05 23:26:35,431 INFO sqlalchemy.engine.base.Engine ('Financial times', 'State-owned banks’ losses widened nearly 3.5 times to Rs 4,284 cr in July-September', 'JP Morgan’s challenge to the European banks in their traditional stronghold is another example of the U.S. bank using its clout to try to take busine ... (47589 characters truncated) ...  services including banking and telephony and became a one-point solution to complete the know your customer (KYC) requirement at an affordable cost.')
2018-12-05 23:26:35,434 INFO sqlalchemy.engine.base.Engine COMMIT

SUMMARY
------------------------------
As per the financial results posted by them, the biggest loss was posted by scam-hit Punjab National Bank (PNB) Rs 4,532.35 crore in the September quarter of the current fiscal as against a profit of Rs 560.58 crore in the year-ago period.
The India’s second largest PSU bank suffered huge losses after the detection of a massive fraud to the tune of nearly Rs 14,000 crore in March quart

2018-12-05 23:26:38,567 INFO sqlalchemy.engine.base.Engine ('Financial times', 'Ashok Chawla resigns as Yes Bank chairman', 'JP Morgan’s challenge to the European banks in their traditional stronghold is another example of the U.S. bank using its clout to try to take busine ... (51882 characters truncated) ... oan books of NBFCs and housing lenders have grown at a compounded annual rate of 17 percent in the past two years, compared with 8 percent for banks.')
2018-12-05 23:26:38,593 INFO sqlalchemy.engine.base.Engine COMMIT

SUMMARY
------------------------------
The bank also said that Independent Director Vasant Gujarathi too has tendered his resignation with immediate effect due to personal commitments.
“Yes Bank announces that Shri Ashok Chawla, Non-Executive Independent Part-Time Chairman, has tendered his resignation from the Bank’s Board, with immediate effect, mentioning that during the current transition period, the Bank would need a Chairman who could devote more time and att

2018-12-05 23:26:41,235 INFO sqlalchemy.engine.base.Engine ('Financial times', 'IL&FS crisis aftermath: Steps for asset monetisation initiated', 'JP Morgan’s challenge to the European banks in their traditional stronghold is another example of the U.S. bank using its clout to try to take busine ... (55763 characters truncated) ...  the Federal Reserve did during the 2008 crisis, when it expanded its historic role and chose to become a lender of last resort even to shadow banks.')
2018-12-05 23:26:41,261 INFO sqlalchemy.engine.base.Engine COMMIT

SUMMARY
------------------------------
The announcement means that an agreement struck in June by IL&FS’ erstwhile board to sell ISSL to IndusInd Bank is nullified, and the process of disposing of the subsidiary will start afresh.
The announcement means that an agreement struck in June by IL&FS’ erstwhile board to sell ISSL to IndusInd Bank is nullified, and the process of disposing of the subsidiary will start afresh.
IL&FS said any agreement 

2018-12-05 23:26:44,850 INFO sqlalchemy.engine.base.Engine COMMIT

SUMMARY
------------------------------
NBFCs push credit demand by 14.4% to a five-year high

Despite the overall increase in lending rates, for first time in over five years, bank credit rose by a healthy 14.41 per cent during the fortnight to October 26, the Reserve Bank has said.
READ ALSO | NBFC crisis poses more growth headwinds, says report

According to reports, NBFCs are finding it difficult to secure short term funding from the money markets and are increasingly looking up to banks for their liabilities.
Apart from that, analysts say liquidity problems at NBFCs offer an opportunity for the banks for recoup their market shares by lending directly to the segments served by the shadow banks.
2018-12-05 23:26:45,137 INFO sqlalchemy.engine.base.Engine INSERT INTO news ("paperName", title, summary) VALUES (?, ?, ?)
2018-12-05 23:26:45,138 INFO sqlalchemy.engine.base.Engine ('Financial times', 'Rupay shines! From limi

IndexError: list index out of range